In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q transformers
!pip install -q datasets
!pip install -q evaluate
!pip install -q scikit-learn
!pip install -q optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==

In [ ]:
from datasets import load_dataset, DatasetDict, IterableDataset
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import torch
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
import optuna
from sklearn.metrics import f1_score


In [ ]:
# Load Grocery and Gourmet Food reviews
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Grocery_and_Gourmet_Food", trust_remote_code=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/30.3k [00:00<?, ?B/s]

Amazon-Reviews-2023.py:   0%|          | 0.00/39.6k [00:00<?, ?B/s]

Grocery_and_Gourmet_Food.jsonl:   0%|          | 0.00/5.97G [00:00<?, ?B/s]

Generating full split: 0 examples [00:00, ? examples/s]

In [ ]:
def split_dataset(dataset, train_ratio=0.8, val_ratio=0.1):
    # Access the 'full' dataset within the DatasetDict
    full_dataset = dataset['full']
    total_samples = len(full_dataset)
    train_size = int(total_samples * train_ratio*0.001)
    val_size = int(total_samples * val_ratio*0.001)

    # Use select on the 'full' dataset
    train_set = full_dataset.select(range(train_size))
    val_set = full_dataset.select(range(train_size, train_size + val_size))
    test_set = full_dataset.select(range(train_size + val_size, train_size + val_size+val_size))

    # Create a new DatasetDict with the split datasets
    split_dataset = DatasetDict({
        'train': train_set,
        'validation': val_set,
        'test': test_set
    })

    return split_dataset['train'], split_dataset['validation'], split_dataset['test']


    # Split the dataset (full size)
train_d, val_d, test_d = split_dataset(dataset)

print(f"Train size: {len(train_d)}, Validation size: {len(val_d)}, Test size: {len(test_d)}")



Train size: 11454, Validation size: 1431, Test size: 1431


In [ ]:
# Convert star ratings to binary labels, removing 3-star reviews
def label_sentiment(example):
    rating = example["rating"]
    if rating == 3:
        example["label"] = -1  # Mark neutral reviews with -1 or another unused label
    else:
        example["label"] = 1 if rating >= 4 else 0  # 4-5 → Positive (1), 1-2 → Negative (0)
    return example

# Apply sentiment labeling and remove neutral reviews
train_dataset = train_d.map(label_sentiment).filter(lambda x: x['label'] != -1) # Filter out the neutral reviews
val_dataset = val_d.map(label_sentiment).filter(lambda x: x['label'] != -1)
test_dataset = test_d.map(label_sentiment).filter(lambda x: x['label'] != -1)


Map:   0%|          | 0/11454 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11454 [00:00<?, ? examples/s]

Map:   0%|          | 0/1431 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1431 [00:00<?, ? examples/s]

Map:   0%|          | 0/1431 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1431 [00:00<?, ? examples/s]

In [ ]:
# Load tokenizer for DistilBert
model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

MAX_SEQUENCE_LENGTH = 50

# Function to preprocess (tokenize) data
def tokenize_function(example):
    review_text = example['text']
    encoded = tokenizer.batch_encode_plus(
            review_text,
            max_length=MAX_SEQUENCE_LENGTH,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_tensors="pt"
        )

    return encoded

# Apply Tokenization to each dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/10577 [00:00<?, ? examples/s]

Map:   0%|          | 0/1332 [00:00<?, ? examples/s]

Map:   0%|          | 0/1331 [00:00<?, ? examples/s]

In [ ]:
###Start Model, Let's fine-tuning the pre-trained model with food dataset

# Load DistilBert model for binary classification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    run_name="food_experiment",
    output_dir='/content/drive/MyDrive/BERT_results',  # Save model checkpoints
    report_to="none",
    evaluation_strategy="epoch",  # Evaluate after each epoch
    save_strategy="epoch",  # Save checkpoints
    per_device_train_batch_size=16,  # Adjust batch size if needed
    per_device_eval_batch_size=16,
    num_train_epochs=2,  # Set number of epochs
    logging_dir="./logs",  # Log directory
    logging_steps=500,
    load_best_model_at_end=True,  # Keep best model based on validation loss
    metric_for_best_model="accuracy",  # Optimize for accuracy
    save_total_limit=2,  # Limit saved checkpoints
)


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    accuracy = (predictions == labels).mean()  # Accuracy calculation
    f1 = f1_score(labels, predictions)  # F1 score calculation
    return {"accuracy": accuracy, "f1": f1}


# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Start training
trainer.train()

# Evaluate on the test set
test_results = trainer.evaluate(eval_dataset=test_dataset)
print(f"Test results for intial model: {test_results}")

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-8-681f2db0be33>:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.212600,0.128831,0.959459,0.978067
2,0.112300,0.107253,0.969219,0.983408


Test results for intial model: {'eval_loss': 0.16622310876846313, 'eval_accuracy': 0.9511645379413974, 'eval_f1': 0.9714034315882094, 'eval_runtime': 5.7832, 'eval_samples_per_second': 230.15, 'eval_steps_per_second': 14.525, 'epoch': 2.0}


In [ ]:
import os
# Fine Tune with Grid Search for hyperparameter
# Define objective function for hyperparameter search
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/BERT_results/checkpoint-1324')

def objective(trial):
    # Define hyperparameters to optimize
    training_args = TrainingArguments(
        output_dir=f'/content/drive/MyDrive/BERT_results/trial_{trial.number}',
        report_to="none",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=trial.suggest_categorical('batch_size', [16, 32]),
        per_device_eval_batch_size=trial.suggest_categorical('batch_size', [16, 32]),
        num_train_epochs=trial.suggest_int('num_train_epochs', 3, 5),
        learning_rate=trial.suggest_loguniform('learning_rate', 5e-6, 5e-5),
        warmup_steps=trial.suggest_int('warmup_steps', 0, 500),
        logging_dir="./logs",
        logging_steps=500,
        load_best_model_at_end=False,
        metric_for_best_model="accuracy",  # Opt for best model based on accuracy
        save_total_limit=5,
    )

    # Define Trainer with the model and datasets
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Start training and evaluate
    trainer.train()

    best_objective_value = 0.0  # Initialize best objective value
    best_epoch = 0

        # Save model parameters after each epoch
    for epoch in range(training_args.num_train_epochs):
        checkpoint_dir = os.path.join(training_args.output_dir, f"checkpoint-{epoch + 1}")
        trainer.save_model(checkpoint_dir)  # Save model and tokenizer

        # Evaluate for the current epoch
        eval_result = trainer.evaluate()
        accuracy = eval_result['eval_accuracy']
        f1 = eval_result['eval_f1']
        objective_value = (accuracy + f1) / 2

        # Update best objective value if current epoch is better
        if objective_value > best_objective_value:
            best_objective_value = objective_value
            best_epoch = epoch + 1

    # You can print or log information about the best epoch here:
    print(f"Trial {trial.number}: Best epoch = {best_epoch}, Objective value = {best_objective_value}")

    # Return the best objective value found across all epochs
    return best_objective_value

# Hyperparameter optimization using Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=3)

[I 2025-04-03 21:43:22,452] A new study created in memory with name: no-name-915c5429-99f5-41cf-ab05-402d5dd95561
<ipython-input-9-facaefb3b96a>:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate=trial.suggest_loguniform('learning_rate', 5e-6, 5e-5),
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-facaefb3b96a>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.062700,0.157319,0.967718,0.982584
2,0.044300,0.169300,0.960210,0.978464
3,0.058100,0.184889,0.970721,0.984293
4,0.019300,0.237017,0.963213,0.980154
5,0.014100,0.234671,0.964715,0.980949


[I 2025-04-03 21:59:47,987] Trial 0 finished with value: 0.9728316175924607 and parameters: {'batch_size': 16, 'num_train_epochs': 5, 'learning_rate': 3.028383038007248e-05, 'warmup_steps': 474}. Best is trial 0 with value: 0.9728316175924607.


Trial 0: Best epoch = 1, Objective value = 0.9728316175924607


<ipython-input-9-facaefb3b96a>:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate=trial.suggest_loguniform('learning_rate', 5e-6, 5e-5),
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-facaefb3b96a>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


...

Trial 1: Best epoch = 1, Objective value = 0.9722575087242835


<ipython-input-9-facaefb3b96a>:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate=trial.suggest_loguniform('learning_rate', 5e-6, 5e-5),
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-facaefb3b96a>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.003100,0.327740,0.962462,0.979691
2,0.004300,0.285383,0.968468,0.983065
3,0.004400,0.383020,0.958709,0.977615
4,0.002600,0.364977,0.960961,0.978862
5,0.001800,0.393073,0.960961,0.978845


[I 2025-04-03 22:24:45,227] Trial 2 finished with value: 0.9699027750288938 and parameters: {'batch_size': 16, 'num_train_epochs': 5, 'learning_rate': 9.387422427217343e-06, 'warmup_steps': 379}. Best is trial 0 with value: 0.9728316175924607.


Trial 2: Best epoch = 1, Objective value = 0.9699027750288938


In [ ]:
# ###Evaluation on best model
# Load the best model from the best trial's checkpoint

# Initial trained model 1324 is the best

checkpoint_path = f'/content/drive/MyDrive/BERT_results/checkpoint-1324'
best_model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)

# Create TrainingArguments with the best parameters
best_training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/BERT_results/final_model',
    report_to="none",
    per_device_eval_batch_size=16,
    metric_for_best_model="accuracy"
)

# Define Trainer with the best model and datasets
trainer = Trainer(
    model=best_model,
    args=best_training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Evaluate on the test set
test_results = trainer.evaluate(eval_dataset=test_dataset)
print(f"Test results for best trial: {test_results}")

# Save the final best model
trainer.save_model()


<ipython-input-11-1eb437b67a2f>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Test results for best trial: {'eval_loss': 0.16622310876846313, 'eval_model_preparation_time': 0.0055, 'eval_accuracy': 0.9511645379413974, 'eval_f1': 0.9714034315882094, 'eval_runtime': 10.7295, 'eval_samples_per_second': 124.05, 'eval_steps_per_second': 7.829}


# Final Model has been created and selected. Now I will fine-tune the model with Movie and TV review data from Amazon Review 2023. I will split new dataset into train, val, and test.

In [ ]:
# Load Movie and TV  reviews
MT_dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Movies_and_TV", trust_remote_code=True)

def split_dataset(dataset, train_ratio=0.8, val_ratio=0.1):
    # Access the 'full' dataset within the DatasetDict
    full_dataset = dataset['full']
    total_samples = len(full_dataset)
    train_size = int(total_samples * train_ratio*0.001)
    val_size = int(total_samples * val_ratio*0.001)

    # Use select on the 'full' dataset
    train_set = full_dataset.select(range(train_size))
    val_set = full_dataset.select(range(train_size, train_size + val_size))
    test_set = full_dataset.select(range(train_size + val_size, train_size + val_size+val_size))

    # Create a new DatasetDict with the split datasets
    split_dataset = DatasetDict({
        'train': train_set,
        'validation': val_set,
        'test': test_set
    })

    return split_dataset['train'], split_dataset['validation'], split_dataset['test']


    # Split the dataset (full size)
MT_train_d, MT_val_d, MT_test_d = split_dataset(MT_dataset)

print(f"Train size: {len(MT_train_d)}, Validation size: {len(MT_val_d)}, Test size: {len(MT_test_d)}")

# Convert star ratings to binary labels, removing 3-star reviews
def label_sentiment(example):
    rating = example["rating"]
    if rating == 3:
        example["label"] = -1  # Mark neutral reviews with -1 or another unused label
    else:
        example["label"] = 1 if rating >= 4 else 0  # 4-5 → Positive (1), 1-2 → Negative (0)
    return example

# Apply sentiment labeling and remove neutral reviews
MT_train_dataset = MT_train_d.map(label_sentiment).filter(lambda x: x['label'] != -1) # Filter out the neutral reviews
MT_val_dataset = MT_val_d.map(label_sentiment).filter(lambda x: x['label'] != -1)
MT_test_dataset = MT_test_d.map(label_sentiment).filter(lambda x: x['label'] != -1)

# Load tokenizer for DistilBert
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
MAX_SEQUENCE_LENGTH = 50

# Function to preprocess (tokenize) data
def tokenize_function(example):
    review_text = example['text']
    encoded = tokenizer.batch_encode_plus(
            review_text,
            max_length=MAX_SEQUENCE_LENGTH,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_tensors="pt"
        )

    return encoded

# Apply Tokenization to each dataset
MT_train_dataset = MT_train_dataset.map(tokenize_function, batched=True)
MT_val_dataset = MT_val_dataset.map(tokenize_function, batched=True)
MT_test_dataset = MT_test_dataset.map(tokenize_function, batched=True)


Movies_and_TV.jsonl:   0%|          | 0.00/8.39G [00:00<?, ?B/s]

Generating full split: 0 examples [00:00, ? examples/s]

Train size: 13862, Validation size: 1732, Test size: 1732


Map:   0%|          | 0/13862 [00:00<?, ? examples/s]

Filter:   0%|          | 0/13862 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/1732 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/12672 [00:00<?, ? examples/s]

Map:   0%|          | 0/1538 [00:00<?, ? examples/s]

Map:   0%|          | 0/1542 [00:00<?, ? examples/s]

In [ ]:
model_path='/content/drive/MyDrive/BERT_results/final_model'
Best_food_model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    accuracy = (predictions == labels).mean()  # Accuracy calculation
    f1 = f1_score(labels, predictions)  # F1 score calculation
    return {"accuracy": accuracy, "f1": f1}



In [ ]:
# Fine Tune with Grid Search for hyperparameter
# Define objective function for hyperparameter search

def objective(trial):
    # Define hyperparameters to optimize
    training_args = TrainingArguments(
        output_dir=f'/content/drive/MyDrive/BERT_results/MT_trial_{trial.number}',
        report_to="none",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=trial.suggest_categorical('batch_size', [16, 32]),
        per_device_eval_batch_size=trial.suggest_categorical('batch_size', [16, 32]),
        num_train_epochs=trial.suggest_int('num_train_epochs', 3, 5),
        learning_rate=trial.suggest_loguniform('learning_rate', 5e-6, 5e-5),
        warmup_steps=trial.suggest_int('warmup_steps', 0, 500),
        logging_dir="./logs",
        logging_steps=500,
        load_best_model_at_end=False,
        metric_for_best_model="accuracy",  # Opt for best model based on accuracy
        save_total_limit=5,
    )

    # Define Trainer with the model and datasets
    trainer = Trainer(
        model=Best_food_model,
        args=training_args,
        train_dataset=MT_train_dataset,
        eval_dataset=MT_val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Start training and evaluate
    trainer.train()
    eval_result = trainer.evaluate()
    accuracy = eval_result['eval_accuracy']
    f1 = eval_result['eval_f1']

    # Return a combination of both metrics (e.g., weighted average)
    # You can use different strategies to combine metrics, here is a simple average:

    # Combine accuracy and F1 into a single objective value (e.g., average)
    objective_value = (accuracy + f1) / 2

    return objective_value  # Return a single float value

# Hyperparameter optimization using Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=3)

[I 2025-04-05 23:02:22,934] A new study created in memory with name: no-name-52578295-1c25-4f28-b358-7f9803b44fcd
<ipython-input-14-57f311bd9773>:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate=trial.suggest_loguniform('learning_rate', 5e-6, 5e-5),
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-14-57f311bd9773>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.162100,0.153516,0.942783,0.967091
2,0.111600,0.160533,0.951235,0.972150
3,0.085500,0.204298,0.949285,0.971260
4,0.052600,0.261659,0.940832,0.965930


[I 2025-04-05 23:18:12,334] Trial 0 finished with value: 0.9533813064173838 and parameters: {'batch_size': 16, 'num_train_epochs': 4, 'learning_rate': 9.287707363384204e-06, 'warmup_steps': 500}. Best is trial 0 with value: 0.9533813064173838.
<ipython-input-14-57f311bd9773>:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate=trial.suggest_loguniform('learning_rate', 5e-6, 5e-5),
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-14-57f311bd9773>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.052000,0.300736,0.945384,0.968820
2,0.032300,0.417069,0.918726,0.952490
3,0.024300,0.357643,0.946684,0.969719
4,0.015600,0.398209,0.938231,0.964433
5,0.021000,0.406797,0.940182,0.965620


[I 2025-04-05 23:36:50,707] Trial 1 finished with value: 0.9529011917327066 and parameters: {'batch_size': 16, 'num_train_epochs': 5, 'learning_rate': 8.398326371437849e-06, 'warmup_steps': 339}. Best is trial 0 with value: 0.9533813064173838.
<ipython-input-14-57f311bd9773>:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate=trial.suggest_loguniform('learning_rate', 5e-6, 5e-5),
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-14-57f311bd9773>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.038800,0.313481,0.945384,0.969231
2,0.051100,0.377410,0.931730,0.960452
3,0.027100,0.360196,0.942783,0.967213
4,0.010300,0.393587,0.946034,0.969018


[I 2025-04-05 23:51:56,353] Trial 2 finished with value: 0.9575260502749556 and parameters: {'batch_size': 16, 'num_train_epochs': 4, 'learning_rate': 3.1875221632163044e-05, 'warmup_steps': 426}. Best is trial 2 with value: 0.9575260502749556.


In [ ]:

# ###Evaluation on best model
# Load the best model from the best trial's checkpoint

# Found trail 0, epoch=2, batch size=16 is the best
checkpoint_path = f'/content/drive/MyDrive/BERT_results/MT_trial_0/checkpoint-1584'
best_food_MT_model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)

# Create TrainingArguments with the best parameters
best_training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/BERT_results/final_food_MT_model',
    report_to="none",
    per_device_eval_batch_size=16,
    metric_for_best_model="accuracy"
)

# Define Trainer with the best model and datasets
trainer = Trainer(
    model=best_food_MT_model,
    args=best_training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Evaluate on the test set
MT_test_results = trainer.evaluate(eval_dataset=MT_test_dataset)
print(f"Test results for best trial: {MT_test_results}")

# Save the final best model
trainer.save_model()

<ipython-input-15-24403305eaad>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Test results for best trial: {'eval_loss': 0.295761376619339, 'eval_model_preparation_time': 0.003, 'eval_accuracy': 0.9111543450064851, 'eval_f1': 0.9510888968225634, 'eval_runtime': 6.6923, 'eval_samples_per_second': 230.414, 'eval_steps_per_second': 14.494}


# In validation stage,

In [ ]:
# Load Movie and TV  reviews
CDV_dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_CDs_and_Vinyl", trust_remote_code=True)

def split_dataset(dataset, train_ratio=0.8, val_ratio=0.1):
    # Access the 'full' dataset within the DatasetDict
    full_dataset = dataset['full']
    total_samples = len(full_dataset)
    train_size = int(total_samples * train_ratio*0.001)
    val_size = int(total_samples * val_ratio*0.001)

    # Use select on the 'full' dataset
    train_set = full_dataset.select(range(train_size))
    val_set = full_dataset.select(range(train_size, train_size + val_size))
    test_set = full_dataset.select(range(train_size + val_size, train_size + val_size+val_size))

    # Create a new DatasetDict with the split datasets
    split_dataset = DatasetDict({
        'train': train_set,
        'validation': val_set,
        'test': test_set
    })

    return split_dataset['train'], split_dataset['validation'], split_dataset['test']


    # Split the dataset (full size)
CDV_train_d, CDV_val_d, CDV_test_d = split_dataset(CDV_dataset)

print(f"Train size: {len(CDV_train_d)}, Validation size: {len(CDV_val_d)}, Test size: {len(CDV_test_d)}")

# Convert star ratings to binary labels, removing 3-star reviews
def label_sentiment(example):
    rating = example["rating"]
    if rating == 3:
        example["label"] = -1  # Mark neutral reviews with -1 or another unused label
    else:
        example["label"] = 1 if rating >= 4 else 0  # 4-5 → Positive (1), 1-2 → Negative (0)
    return example

# Apply sentiment labeling and remove neutral reviews
CDV_train_dataset = CDV_train_d.map(label_sentiment).filter(lambda x: x['label'] != -1) # Filter out the neutral reviews
CDV_val_dataset = CDV_val_d.map(label_sentiment).filter(lambda x: x['label'] != -1)
CDV_test_dataset = CDV_test_d.map(label_sentiment).filter(lambda x: x['label'] != -1)

# Load tokenizer for RoBERTa
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base" )

MAX_SEQUENCE_LENGTH = 50

# Function to preprocess (tokenize) data
def tokenize_function(example):
    review_text = example['text']
    encoded = tokenizer.batch_encode_plus(
            review_text,
            max_length=MAX_SEQUENCE_LENGTH,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_tensors="pt"
        )

    return encoded

# Apply Tokenization to each dataset
CDV_train_dataset = CDV_train_dataset.map(tokenize_function, batched=True)
CDV_val_dataset = CDV_val_dataset.map(tokenize_function, batched=True)
CDV_test_dataset = CDV_test_dataset.map(tokenize_function, batched=True)


CDs_and_Vinyl.jsonl:   0%|          | 0.00/3.29G [00:00<?, ?B/s]

Generating full split: 0 examples [00:00, ? examples/s]

Train size: 3861, Validation size: 482, Test size: 482


Map:   0%|          | 0/3861 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3861 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/482 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/3676 [00:00<?, ? examples/s]

Map:   0%|          | 0/468 [00:00<?, ? examples/s]

Map:   0%|          | 0/455 [00:00<?, ? examples/s]

In [ ]:
# ###Evaluation on best model without further fine-tuning
# Load the best model from the best trial's checkpoint
checkpoint_path = f'/content/drive/MyDrive/BERT_results/MT_trial_0/checkpoint-1584'
best_food_MT_model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)

# Create TrainingArguments with the best parameters
best_training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/BERT_results/final_food_MT_model',
    report_to="none",
    per_device_eval_batch_size=16,
    metric_for_best_model="accuracy"
)

# Define Trainer with the best model and datasets
trainer = Trainer(
    model=best_food_MT_model,
    args=best_training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Evaluate on the test set
CDV_test_results = trainer.evaluate(eval_dataset=CDV_test_dataset)
print(f"Test results for best trial: {CDV_test_results}")

# Save the final best model
trainer.save_model()

<ipython-input-17-568c3bc538b2>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Test results for best trial: {'eval_loss': 0.10296513140201569, 'eval_model_preparation_time': 0.0054, 'eval_accuracy': 0.9648351648351648, 'eval_f1': 0.9811764705882353, 'eval_runtime': 2.0768, 'eval_samples_per_second': 219.087, 'eval_steps_per_second': 13.964}
